In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import glob
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import time
import cv2
from sklearn.model_selection import train_test_split
import gc
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation,Dropout,BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train', 'test', 'sample_submission.csv']


In [46]:
data = '../input/'
train_files = sorted(glob.glob(data + 'train/*/*'))
# test_path = sorted(glob.glob(data + 'test/*'))

labels = ['HTC-1-M7', 'iPhone-4s', 'iPhone-6', 'LG-Nexus-5x', 'Motorola-Droid-Maxx', 
         'Motorola-Nexus-6', 'Motorola-X', 'Samsung-Galaxy-Note3', 'Samsung-Galaxy-S4',
         'Sony-NEX-7'] 
All_labels = [labels.index(file.split('/')[-2]) for file in train_files] # TO BE CHANGED!!!!!!!

x_train_files, x_test_files, y_train_files, y_test_files = train_test_split(train_files,All_labels,test_size = 0.05, shuffle = True,random_state = 42)

# y_train = keras.utils.to_categorical(y_train_files, len(labels)) # one hot vector
y_test = keras.utils.to_categorical(y_test_files, len(labels)) # one hot vector

In [39]:
# y_train_files
len(y_train_files)

137

In [40]:
crdim = 256

kernel_filter = 1/12. * np.array([\
            [-1,  2,  -2,  2, -1],  \
            [ 2, -6,   8, -6,  2],  \
            [-2,  8, -12,  8, -2],  \
            [ 2, -6,   8, -6,  2],  \
            [-1,  2,  -2,  2, -1]])






train_imgs = []
count_file = -1
y_augm = [] # augmented labels
for file in tqdm_notebook(x_train_files):
    img = cv2.imread(file)
    x,y = img.shape[:2]

    count_file += 1 # number of img out of 2750 (or len of train set)
    img_label = y_train_files[count_file]    
    

    x,y = img.shape[:2]

    xpos = []
    xpos.append([0,crdim])
    xpos.append([x//2-crdim//2,x//2+crdim//2])
#     xpos.append([x-crdim,x])

    ypos = []
    ypos.append([0,crdim])
    ypos.append([y//2-crdim//2,y//2+crdim//2])
#     ypos.append([y-crdim,y])

    cr_imgs = []
    for i in xpos:
        for j in ypos:
            crimg = img[i[0]:i[1], j[0]:j[1],:]
            crimg = cv2.filter2D(crimg.astype(np.float32),-1,kernel_filter)
            train_imgs.append(crimg)
            
    for t in range(len(xpos)*len(ypos)): # total crops
        y_augm.append(y_train_files[count_file]) # augment labels

            
    del file,img,crimg
    gc.collect()

train_imgs = np.array(train_imgs)

In [41]:
y_train = keras.utils.to_categorical(y_augm, len(labels)) # one hot vector

In [43]:
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip = True)

datagen.fit(train_imgs)

In [44]:
model=Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),strides=(2, 2), input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, kernel_size=(3, 3),strides=(2, 2) ))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, kernel_size=(3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(10))
model.add(Activation('softmax'))

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 127, 127, 64)      1792      
_________________________________________________________________
batch_normalization_4 (Batch (None, 127, 127, 64)      256       
_________________________________________________________________
activation_7 (Activation)    (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 63, 63, 64)        36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 63, 63, 64)        256       
_________________________________________________________________
activation_8 (Activation)    (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 61, 61, 32)        18464     
__________

In [45]:
model.fit_generator(datagen.flow(train_imgs, y_train, batch_size=32),steps_per_epoch= None, epochs=100) 

Epoch 1/100
18/18 [==============================] - 6s 333ms/step - loss: 3.5223 - acc: 0.1236
Epoch 2/100
18/18 [==============================] - 2s 101ms/step - loss: 2.2740 - acc: 0.1926
Epoch 3/100
18/18 [==============================] - 2s 100ms/step - loss: 2.2620 - acc: 0.1480
Epoch 4/100
18/18 [==============================] - 2s 102ms/step - loss: 2.2346 - acc: 0.1839
Epoch 5/100
18/18 [==============================] - 2s 100ms/step - loss: 2.1664 - acc: 0.1682
Epoch 6/100
18/18 [==============================] - 2s 100ms/step - loss: 2.1864 - acc: 0.1891
Epoch 7/100
18/18 [==============================] - 2s 101ms/step - loss: 2.1952 - acc: 0.2152
Epoch 8/100
18/18 [==============================] - 2s 101ms/step - loss: 2.1256 - acc: 0.2065
Epoch 9/100
18/18 [==============================] - 2s 101ms/step - loss: 2.1969 - acc: 0.2141
Epoch 10/100
18/18 [==============================] - 2s 101ms/step - loss: 2.1478 - acc: 0.2019
Epoch 11/100
18/18 [===================

18/18 [==============================] - 2s 101ms/step - loss: 0.9430 - acc: 0.6835
Epoch 86/100
18/18 [==============================] - 2s 101ms/step - loss: 0.8203 - acc: 0.7176
Epoch 87/100
18/18 [==============================] - 2s 101ms/step - loss: 0.8307 - acc: 0.6887
Epoch 88/100
18/18 [==============================] - 2s 102ms/step - loss: 0.7752 - acc: 0.7354
Epoch 89/100
18/18 [==============================] - 2s 101ms/step - loss: 0.7116 - acc: 0.7528
Epoch 90/100
18/18 [==============================] - 2s 103ms/step - loss: 0.8654 - acc: 0.7252
Epoch 91/100
18/18 [==============================] - 2s 101ms/step - loss: 0.7734 - acc: 0.7246
Epoch 92/100
18/18 [==============================] - 2s 101ms/step - loss: 0.8736 - acc: 0.7263
Epoch 93/100
18/18 [==============================] - 2s 102ms/step - loss: 0.7743 - acc: 0.7413
Epoch 94/100
18/18 [==============================] - 2s 102ms/step - loss: 0.6736 - acc: 0.7779
Epoch 95/100
18/18 [=======================

In [47]:
dim = 256

kernel_filter = 1/12. * np.array([\
            [-1,  2,  -2,  2, -1],  \
            [ 2, -6,   8, -6,  2],  \
            [-2,  8, -12,  8, -2],  \
            [ 2, -6,   8, -6,  2],  \
            [-1,  2,  -2,  2, -1]])


test_imgs = []
for file in tqdm_notebook(x_test_files):
    img = cv2.imread(file)
    center_x = img.shape[0]//2
    center_y = img.shape[1]//2 
    cropped_img = img[center_x-dim//2:center_x+dim//2, center_y-dim//2:center_y+dim//2, : ]
    
    cropped_img = cv2.filter2D(cropped_img.astype(np.float32),-1,kernel_filter)    
    test_imgs.append(cropped_img)
    del file,img,center_x,center_y,cropped_img
    gc.collect()

test_imgs = np.array(test_imgs)

In [ ]:
scores = model.evaluate(test_imgs,y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

128/138 [==========================>...] - ETA: 0s